In [9]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf

In [20]:
import pickle
# load
with open('../generator/dataset_optiver_ANN.pickle', 'rb') as f:
    data = pickle.load(f)
    
raw_data = pd.read_csv('../train.csv')

In [10]:
X_train = data['X_train']
np.nan_to_num(X_train, copy=False)
X_val = data['X_val']
np.nan_to_num(X_val, copy=False)
X_test = data['X_test']
np.nan_to_num(X_test, copy=False)
y_train = data['y_train']
np.nan_to_num(y_train, copy=False)
y_val = data['y_val']
np.nan_to_num(y_val, copy=False)
y_test = data['y_test']
np.nan_to_num(y_test, copy=False)

# NumPy 배열을 TensorFlow 텐서로 변환
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_val = tf.convert_to_tensor(X_val)
y_val = tf.convert_to_tensor(y_val)
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

In [12]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# 가정: X는 3D 데이터 텐서
num_train = len(X_train)
num_val = len(X_val)
num_test = len(X_test)
data_shape = (200, 12)

# tf.reshape를 사용하여 데이터를 2D 행렬로 변환
X_train_reshaped = tf.reshape(X_train, (num_train, -1))#[:-2700]
y_train_reshaped = tf.reshape(y_train, (num_train, -1))#[:-2700]
X_val_reshaped = tf.reshape(X_val, (num_val, -1))
y_val_reshaped = tf.reshape(y_val, (num_val, -1))
X_test_reshaped = tf.reshape(X_test, (num_test, -1))
y_test_reshaped = tf.reshape(y_test, (num_test, -1))


In [15]:
X_train_reshaped[-1]

<tf.Tensor: shape=(1600,), dtype=float64, numpy=
array([1.000884  , 0.00345425, 1.000884  , ..., 0.00754767, 1.00309   ,
       0.70737497])>

In [27]:
for i in X_val_reshaped[0]:
    print(i)

tf.Tensor(0.999288, shape=(), dtype=float64)
tf.Tensor(0.003658384482721818, shape=(), dtype=float64)
tf.Tensor(0.999288, shape=(), dtype=float64)
tf.Tensor(0.00011864794803600557, shape=(), dtype=float64)
tf.Tensor(0.999385, shape=(), dtype=float64)
tf.Tensor(0.0011288302668138932, shape=(), dtype=float64)
tf.Tensor(0.999293, shape=(), dtype=float64)
tf.Tensor(0.7078411997227548, shape=(), dtype=float64)
tf.Tensor(0.999642, shape=(), dtype=float64)
tf.Tensor(0.0006253425478753965, shape=(), dtype=float64)
tf.Tensor(0.999337, shape=(), dtype=float64)
tf.Tensor(0.0018290390144778184, shape=(), dtype=float64)
tf.Tensor(1.000435, shape=(), dtype=float64)
tf.Tensor(0.002476042486184451, shape=(), dtype=float64)
tf.Tensor(0.999657, shape=(), dtype=float64)
tf.Tensor(0.7084665608313487, shape=(), dtype=float64)
tf.Tensor(1.00382, shape=(), dtype=float64)
tf.Tensor(0.0013427027115274432, shape=(), dtype=float64)
tf.Tensor(1.003769, shape=(), dtype=float64)
tf.Tensor(4.5229373416144865e-05, sh

In [26]:
train = raw_data.query('date_id<478')
train[-2700:]

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
5202280,100,477,410,0.00,0,0.995446,1319486.30,0.995446,0.995446,0.993372,95244.36,0.995446,90744.00,0.994435,-4.249811,26276,477_410_100
5202281,101,477,410,2102588.37,1,0.997947,8668113.87,1.049599,1.006167,0.996757,162718.85,0.998264,64940.90,0.997835,-6.589889,26276,477_410_101
5202282,102,477,410,1074844.12,1,1.001280,25897061.88,1.014544,1.006562,1.001220,100092.00,1.001580,4505.76,1.001565,-11.150241,26276,477_410_102
5202283,103,477,410,3718999.79,-1,1.000737,10264427.91,0.902214,0.975568,1.000737,143.93,1.000946,10077.20,1.000740,-0.900030,26276,477_410_103
5202284,104,477,410,0.00,0,1.001915,21896763.44,1.001915,1.001915,1.001570,228242.30,1.001915,241587.66,1.001738,-3.340244,26276,477_410_104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5204975,195,477,540,2809393.38,-1,1.002359,35296955.75,1.001253,1.001253,1.002359,424927.60,1.002482,326361.57,1.002429,-3.190041,26289,477_540_195
5204976,196,477,540,1236161.74,-1,0.998663,8814158.85,0.996644,0.997401,0.998663,36276.52,0.999168,95031.58,0.998803,-6.200075,26289,477_540_196
5204977,197,477,540,509422.85,1,0.997623,10364257.03,0.998388,0.998388,0.997240,20846.00,0.997623,27527.28,0.997405,0.000000,26289,477_540_197
5204978,198,477,540,5724317.95,-1,0.999832,63738563.90,0.998511,0.998751,0.999712,538354.70,0.999952,967689.35,0.999797,1.300573,26289,477_540_198
